# Try out crystal-tree
This notebook contains some examples on how to use crystal-tree python package for explaining Decision Trees.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

from crystal_tree import CrystalTree, Trace

### Default explanations
To obtain the summarized explanations in the default way, first create a ```CrystalTree()``` object and passing your Decision Tree Classifier object as argument. A call to the method ```explain()``` will return an array of ```Explanation``` objects, just as the typical call to ```predict()``` would return an array of classifications. ```Explanation``` objects are tree-like which come with handy methods like ```ascii_tree()``` to automatically produce the text based explanation.

In [2]:
# Loads dataset
X, y = load_iris(return_X_y=True, as_frame=True)

# Trains decision tree
clf = DecisionTreeClassifier()
clf.fit(X,y)

# Translates the classifier into an explainable logic program
crys_tree = CrystalTree(clf)

# Print explanations of input X (two arbitrary rows)
for e in crys_tree.explain(X.iloc[[0, 54]]):
    print(e.ascii_tree())

  *
  |__Predicted class 0 for instance 0
  |  |__petal width (cm) <= 0.8

  *
  |__Predicted class 1 for instance 1
  |  |__petal length (cm) <= 4.9
  |  |__petal width (cm) in (0.8,1.6]



 #### Manipulating explanations
 Of course, one could manipulate that object to prune part of the explanation, create new brances, or even automatically translate the explanations before printing them.

In [16]:
from translate import Translator

# Loads dataset
X, y = load_iris(return_X_y=True, as_frame=True)

# Trains decision tree
clf = DecisionTreeClassifier()
clf.fit(X,y)

# Translates the classifier into an explainable logic program
crys_tree = CrystalTree(clf)


translator = Translator(to_lang='DE')
for e in crys_tree.explain(X.iloc[[0, 54]]):
    node_iterator = e.preorder_iterator()
    next(node_iterator)  # Skips root node (it has no labels)
    for node, index in node_iterator:  
        for i in range(len(node.labels)):
            node.labels[i] = translator.translate(node.labels[i])
    print(e.ascii_tree())

  *
  |__Vorhergesagte Klasse 0 für Beispiel 0
  |  |__blütenblattlänge (cm) &lt;= 2,4

  *
  |__Vorhergesagte Klasse 1 für Beispiel 1
  |  |__blütenblattbreite (cm) &lt;= 1,6
  |  |__blütenblattlänge (cm) in (2.4,4.9]



#### Custom text labels
Another approach to personalize explanations is to directly create the labels that will be used. It just allow to define new predefined text labels, but when obtaining large explanations, this will be much more computationally efficient than traversing all the produced explanation trees.
The example below demonstrate how to define new text labels for each target class.

In [ ]:
def setup_labels(tree: CrystalTree):
    """Just setups some traces"""
    tree.add_trace(Trace("Predicted iris-virginica", "prediction", target_class=0))
    tree.add_trace(Trace("Predicted iris-versicolor", "prediction", target_class=1))
    tree.add_trace(Trace("Predicted iris-setosa", "prediction", target_class=2))

# Loads dataset
X, y = load_iris(return_X_y=True, as_frame=True)

# Trains decision tree
clf = DecisionTreeClassifier()
clf.fit(X,y)

# Translates the classifier into an explainable logic program
crys_tree = CrystalTree(clf)

# Apply the labels used by the tree
setup_labels(crys_tree)

# Print explanations of input X (first two rows)
for e in crys_tree.explain(X.iloc[[0, 54]]):
    print(e.ascii_tree())